<a href="https://colab.research.google.com/github/angiellanos/MNIST_DF/blob/main/MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Se usan las funciones *_load_label* y *_load_img* para cargar el conjunto de datos del MNIST. Luego, se usa la función *train_test_split* de la librería *sklearn* para dividir el conjunto de datos en un conjunto de entrenamiento y uno de prueba, asegurándose de que ambos estén balanceados. Es decir, que tengan la misma proporción de imágenes para cada dígito. Se usa el parámetro *stratify* de la función *train_test_split* para lograr esto. El tamaño del conjunto de prueba es 10000, entonces el conjunto de prueba tiene 10000 imágenes y el conjunto de entrenamiento 60000 imágenes.

In [1]:
import os.path
import gzip
import pickle
import os
import numpy as np
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Rutas y nombres de los archivos que contienen los datos de MNIST
route = 'http://yann.lecun.com/exdb/mnist/'
files = {
    'train_img':'train-images-idx3-ubyte.gz',
    'train_label':'train-labels-idx1-ubyte.gz',
    'test_img':'t10k-images-idx3-ubyte.gz',
    'test_label':'t10k-labels-idx1-ubyte.gz'
}

# Directorio actual y nombre del archivo donde se guardó el conjunto de datos en formato pickle
dataset_direct = os.getcwd()
save_file = dataset_direct + "/mnist.pkl"

image_size = 28
num_images = [60000, 10000]
num_labels = [60000, 10000]

def _load_label(file_key):
    """Función que carga las etiquetas de las imágenes desde un archivo comprimido en formato gzip.
    Args:
        file_name (str): el nombre del archivo que contiene las etiquetas
    Returns:
        array: etiquetas de las imágenes.
    """
    file_name = files[file_key]
    file_path = dataset_direct + "/" + file_name

    j = 0 if file_key=='train_label' else 1
    with gzip.open(file_path, 'rb') as f:
        # omite los primeros 8 bytes
        f.read(8)
        buf = f.read(num_labels[j])
        label = np.frombuffer(buf, dtype=np.uint8).astype(np.int64)
        # print(label[:20])

    return label

def _load_img(file_key):
    """Función que carga las imágenes desde un archivo comprimido en formato gzip.
    Args:
        file_name (str): el nombre del archivo que contiene las etiquetas
    Returns:
        array: imágenes en forma de vectores de tamaño 784.
    """
    file_name = files[file_key]
    file_path = dataset_direct + "/" + file_name

    j = 0 if file_key=='train_img' else 1
    with gzip.open(file_path, 'rb') as f:
        # omite los primeros 16 bytes
        f.read(16)
        # cada pixel en 1 byte = 8 bits
        # lee todos los datos y los coloca en un buffer de memoria
        buf = f.read(image_size * image_size * num_images[j])
        # traslada los datos a un array de numpy de tipo float32
        data = np.frombuffer(buf, dtype=np.uint8).astype(np.float32)
        # Cambia la forma de los datos par entregarlos listos al dataset
        data = data.reshape(num_images[j], image_size, image_size, 1)

    return data

class ImageDataset(Dataset):
    def __init__(self, labels, images, transform=None, target_transform=None):
        self.labels = labels
        self.images = images
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        image = self.images[idx]
        label =  self.labels[idx]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

def _convert_numpy():
    """Función que convierte los datos del MNIST en arrays de NumPy y los guarda en dos diccionarios.
    Args: None
    Returns:
        dictionary: con las claves ‘train_img’, ‘train_label’, ‘test_img’ y ‘test_label’,
                   y los valores correspondientes a los arrays de NumPy con las imágenes
                   y las etiquetas.
    """
    # Cargar el conjunto de datos del MNIST usando las funciones _load_label y _load_img
    X_train = _load_img('train_img')
    y_train = _load_label('train_label')
    X_test = _load_img('test_img')
    y_test = _load_label('test_label')

    # Dividir el conjunto de datos en un conjunto de entrenamiento y uno de prueba, usando la función train_test_split
    # Se usa el parámetro stratify para asegurar que ambos conjuntos estén balanceados
    X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=10000, stratify=y_train)

    # Crear los datasets personalizados usando la clase ImageDataset
    train_dataset = ImageDataset(y_train, X_train)
    test_dataset = ImageDataset(y_test, X_test)

    # Imprimir las dimensiones de los conjuntos de datos
    print("Dimensiones del conjunto de entrenamiento:")
    print(X_train.shape)
    print(y_train.shape)
    print("Dimensiones del conjunto de prueba:")
    print(X_test.shape)
    print(y_test.shape)

    return train_dataset, test_dataset


train_dataset, test_dataset = _convert_numpy()





## Preparando los datos para entrenamiento con DataLoaders
num_workers = torch.get_num_threads()//2

train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers= num_workers)

test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

print(torch.get_num_threads())

len(next(iter(test_dataloader)))


## Iterando a lo largo del DataLoader

# Despliega imagen y etiqueta
train_features, train_labels = next(iter(train_dataloader))
print(f'Shape del lote de imágenes: {train_features.size()}')
print(f'Shape del lote de etiquetas: {train_labels.size()}')
img = train_features[0].squeeze()# primer elemento del batch
# squeeze elimina ejes de tamaño 1.
label = train_labels[0]
plt.imshow(img, cmap='gray')
plt.title(label.numpy())
plt.show()

class Draw:
    def __init__(self, label_map, images_iterator, cols=4, rows=4, figsize=(6,6)):
        self.data = images_iterator
        self.label_map = label_map
        self.figsize=figsize
        self.cols = cols
        self.rows = rows
        self.index = 0

    def __iter__(self):
        return self

    def __next__(self):
        if self.index >= len(self.data):
            raise StopIteration

        self.index += 1

        img, label = next(self.data)
        figure = plt.figure(figsize=self.figsize)
        for i in range(1, self.cols*self.rows+1):
            #sample_idx = torch.randint(len(training_data), size =(1,)).item()
            #img, label = training_data[sample_idx]
            figure.add_subplot(self.rows, self.cols, i)
            #plt.title(self.label_map[label])
            plt.axis('off')
            plt.imshow(img[i].squeeze(), cmap='gray')
            plt.title(label[i].numpy())
        plt.show()
        #return value

label_map = {
    0: 'camiseta',
    1: "Pantalones",
    2: "Jersey",
    3: "Vestido",
    4: "Abrigo",
    5: "Sandalia",
    6: "Camisa",
    7: "Tenis",
    8: "Bolso",
    9: "Botines",
}

images = Draw(label_map, iter(train_dataloader))

next(images)

FileNotFoundError: ignored